In [58]:
%matplotlib inline
%xmode plain
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import torch.nn as nn
import torch.nn.functional as f
import torch
import torch.optim as optim
import time

Exception reporting mode: Plain


In [59]:
# all_cards = [cv2.imread(file) for file in glob.glob("../CV-BlackjackStudent/GOPR0317.MP4-lbl//*.png")]
def load_suit_data():
    x_train = np.ones((1,60*100))
    y_train = [] 
    suits = ['14w', '15w', '16w', '17w']
    for filename in glob.glob("./training_data//*.png"):

        ## read spades
        for j, substr in enumerate(suits):
            if substr in  filename:
                img = cv2.imread(filename)
                img = cv2.resize(img, (60, 100))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                _,img = cv2.threshold(img, 128, 255, 0)
                img = img.flatten()[np.newaxis, :]
                x_train = np.vstack((x_train, img))
                y_train.append(j)
                
    y_train = np.array([y_train])
    x_train = x_train[1:x_train.shape[1], :]
    
    return x_train, y_train

In [60]:
def load_rank_data():
    x_train = np.ones((1,60*100))
    y_train = [] 
    ranks = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13']
    for filename in glob.glob("./training_data/ranks/*.png"):

        ## read spades
        for j, substr in enumerate(ranks):
            if substr in  filename:
                img = cv2.imread(filename)
                img = cv2.resize(img, (60, 100))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                _,img = cv2.threshold(img, 128, 255, 0)
                img = img.flatten()[np.newaxis, :]
                x_train = np.vstack((x_train, img))
                y_train.append(j)

        ## read hearts 

    y_train = np.array([y_train])
    x_train = x_train[1:x_train.shape[1], :]
    
    return x_train, y_train

In [70]:
x_train, y_train = load_rank_data()
display(x_train.shape, y_train.shape)

(90, 6000)

(1, 90)

In [71]:
y_train = y_train.T
joint_data = np.hstack((x_train, y_train))

In [72]:
np.random.shuffle(joint_data)
x_train = joint_data[:, 0:joint_data.shape[1]-1]
y_train = joint_data[:, -1]

In [74]:
class Net(nn.Module):
    def __init__(self, num_inputs=6000, num_classes=13):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_inputs, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    


In [75]:
t_xtrain = torch.tensor(x_train).type(torch.float32)
t_ytrain = torch.tensor(y_train).type(torch.LongTensor)
t_ytrain

tensor([ 10,   0,   7,  11,  11,   4,  11,   7,   6,   7,   5,   6,
          5,   9,   6,   9,  12,   4,   3,   4,  12,   0,   0,   6,
          6,   0,   6,  12,   4,  10,   6,  11,   0,   7,   4,   6,
          2,   9,   5,   1,   3,   4,  12,   8,   3,   9,   2,   5,
          1,   4,   8,  10,   7,   9,  10,   3,  11,  12,   2,  10,
         10,   9,   5,   3,   1,   7,   8,   1,   3,  11,   8,   0,
          8,   8,   2,   1,   3,   9,  10,   2,  10,   9,   5,  12,
          1,   7,   2,  11,  12,   3])

In [76]:
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.0001)
loss_fn = torch.nn.CrossEntropyLoss()

for i in range(100000):
    optimizer.zero_grad()
    outputs = net(t_xtrain).type(torch.float32) #.max(1)[0].type(torch.float32).unsqueeze(1)
    loss = loss_fn(outputs, t_ytrain)
        
    if i % 500 == 0:
        outputs = net(t_xtrain)
        acc = torch.sum(net(t_xtrain).max(1)[1].reshape(1,x_train.shape[0]) == t_ytrain)
        acc = acc.item()
        print('loss: {:.4f} acc: {:.4f} '.format(loss.data, acc/101))
        
        
    if loss < 0.0001:
        break
    loss.backward()
    optimizer.step()
    

loss: 34.7493 acc: 0.0891 
loss: 0.0017 acc: 0.8911 
loss: 0.0008 acc: 0.8911 
loss: 0.0005 acc: 0.8911 
loss: 0.0004 acc: 0.8911 
loss: 0.0003 acc: 0.8911 
loss: 0.0003 acc: 0.8911 
loss: 0.0002 acc: 0.8911 
loss: 0.0002 acc: 0.8911 
loss: 0.0002 acc: 0.8911 
loss: 0.0001 acc: 0.8911 
loss: 0.0001 acc: 0.8911 
loss: 0.0001 acc: 0.8911 
loss: 0.0001 acc: 0.8911 
loss: 0.0001 acc: 0.8911 


In [67]:
torch.save(net.state_dict(), './suits-model-'+str(time.time())+'.csv')

In [77]:
for i in range(15):
    cv2.imshow(str(y_train[i]),x_train[i].reshape(100,60))
cv2.waitKey(0)
cv2.destroyAllWindows()
